In [2]:
# import packages:

import pandas as pd
import numpy as np
import os
import time as tm
import pandas_datareader as pdr
import yfinance as yf

import datetime
import requests_cache
import random
import glob

pd.set_option('max_rows', 100)
pd.set_option('max_columns', 100)


In [7]:
# functions:

def pull_stocks(stock_list, start_date, end_date, file_save_location, sleep_min = 2, sleep_max = 10, expire_days = 3):
    
    session = requests_cache.CachedSession(cache_name='cache', backend='sqlite', expire_after=expire_days)
    
    for stock in stock_list:
        
        start = tm.time()
        print('retrieving: ' , stock)
    
        ## pull and data generation:
        df = pdr.get_data_yahoo(stock.strip(), start = start_date, end = end_date)
        df['ticker'] = stock
        df = df.reset_index()
        df.to_csv(os.path.join(file_save_location, stock.strip() +'.csv'), index = False)
        print('saving: ', stock, ' data to: ', os.path.join(file_save_location, stock.strip() +'.csv'))

        end = tm.time()
        total = (end - start) / 60
        print(f"completed retrieving: {stock.strip()} data in: {total}")
        
        sleep_time = random.randint(2, 10)
        print(f"sleeping for: {sleep_time} seconds")
       

        # sleep between pulls so to not arouse suspicion:
        tm.sleep(sleep_time)

        del df, start, end, total, sleep_time
        
        
def fix_columns(dataframe: pd.DataFrame) -> list:
    
    '''function that takes a list of columns and modifies them to be easier to read -- assign to df.columns'''
    
    column_string_replace = ['\n','@',' ','__', '/', '-']

    columns = dataframe.columns

    columns = columns.map(lambda x: x.strip())
    columns = columns.map(lambda x : x.lower())

    for string in column_string_replace:
        columns = columns.map(lambda x : x.replace(string, '_') if isinstance (x, (str, bytes)) else x)

    return columns
        

def calculate_rolling_mean(dataframe: pd.DataFrame, field: str, days: int) -> np.array:
    '''Function that calculates a rolling mean for a given period and returns it to be added to the dataframe
    '''
    moving_array =  dataframe[field].rolling(days).mean()
    
    return moving_array



In [8]:
# pull the data down:
'''
start = '2018-01-01'
end = '2020-12-30'
file_location = '../data/01_raw/historic_data'

etfs = ['XLY', 'XLP', 'XLE', 'XLF', 'XLV', 'XLI', 'XLB', 'XLRE', 'XLK', 'XLU', 'SPY']

pull_stocks(stock_list = etfs, start_date = start, end_date = end, file_save_location = file_location)

'''

"\nstart = '2018-01-01'\nend = '2020-12-30'\nfile_location = '../data/01_raw/historic_data'\n\netfs = ['XLY', 'XLP', 'XLE', 'XLF', 'XLV', 'XLI', 'XLB', 'XLRE', 'XLK', 'XLU', 'SPY']\n\npull_stocks(stock_list = etfs, start_date = start, end_date = end, file_save_location = file_location)\n\n"

In [74]:
df = pd.read_csv('../data/01_raw/historic_data/XLU.csv')

df.columns = fix_columns(df)

In [75]:
def calculate_rolling_means(dataframe: pd.DataFrame, field: str, day_ranges: list, exponential = False) -> pd.DataFrame: 
    
    '''return a dataframe that includes all of the rolling means either straight or exponential appended to the overall dataset'''
    
    if exponential == True:
        
        for days in day_ranges:
            dataframe[str(days) +'_' + field + '_' + 'ema'] = dataframe[field].ewm(span = days, min_periods = days).mean()
    
    else:
        for days in day_ranges:
            dataframe[str(days) +'_' + field + '_' + 'sma'] = dataframe[field].rolling(days).mean()
        
    return dataframe

In [76]:
day_ranges = [20, 30]

df = calculate_rolling_means(dataframe = df, field = 'close', day_ranges = day_ranges, exponential = True)
df = calculate_rolling_means(dataframe = df, field = 'close', day_ranges = day_ranges, exponential = False)

In [77]:
# create function to calculate rolling standard deviations:

def calculate_rolling_std(dataframe: pd.DataFrame, field: str, day_ranges: list) -> pd.DataFrame: 
    
    '''return a dataframe that includes all of the rolling standard deviations of a given field'''
    
    for days in day_ranges:
        dataframe[str(days) +'_' + field + '_' + 'std'] = dataframe[field].rolling(days).std()

    return dataframe
 


In [78]:
df = calculate_rolling_std(dataframe = df , field = 'close', day_ranges = day_ranges)

In [79]:
df.head()

,date,high,low,open,close,volume,adj_close,ticker,20_close_ema,30_close_ema,20_close_sma,30_close_sma,20_close_std,30_close_std
0,2018-01-02,52.740002,52.029999,52.669998,52.189999,12270000.0,46.506157,XLU,NaN,NaN,NaN,NaN,NaN,NaN
1,2018-01-03,52.380001,51.599998,52.000000,51.779999,20807900.0,46.140812,XLU,NaN,NaN,NaN,NaN,NaN,NaN
2,2018-01-04,51.959999,51.250000,51.619999,51.349998,14971800.0,45.757637,XLU,NaN,NaN,NaN,NaN,NaN,NaN
3,2018-01-05,51.610001,51.070000,51.529999,51.330002,23416300.0,45.739819,XLU,NaN,NaN,NaN,NaN,NaN,NaN
4,2018-01-08,51.849998,51.320000,51.419998,51.810001,14300500.0,46.167542,XLU,NaN,NaN,NaN,NaN,NaN,NaN
